# Steps to solve Emergency vs Non-Emergency vehicle classification problem

<ol>1. Import necessary libraries</ol>
<ol>2. Loading and pre-processing the Data</ol>
<ol>3. Load weights of pretrained model</ol>
<ol>4. Fine tune the model for the current problem</ol>
<ol>5. Validate if it works fine, iterate again if it does not</ol>

## 1. Import necessary libraries

In [16]:
# importing the required libraries
%matplotlib inline
import numpy as np  
import pandas as pd 
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import Image

# importing libraries for defining the architecture of model
import torch 
import torch.nn as nn
from torch.optim import Adam
from torch.nn import Linear, ReLU, BCELoss, Sequential, Sigmoid

# import torchvision
from torchvision.models import googlenet
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

## 2. Loading and pre-processing the Dataset



In [ ]:
#mounting the drive
from google.colab import drive
drive.mount('/content/drive')  

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#extract the images
!unzip '/content/drive/My Drive/Dataset.zip'

In [ ]:
# defining the pre-processing steps
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

preprocessing = transforms.Compose([transforms.ToTensor(),
                                    normalize,
])

In [ ]:
# defining the class to load dataset 
class EmergencyDataset(Dataset):
    """Custom Dataset for loading Emergency Dataset"""

    # defining the init function
    def __init__(self, csv_path, img_dir, transform):
    
        df = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.image_names.values
        self.y = df['emergency_or_not'].values
        self.transform = transform

    # defining the get item function
    def __getitem__(self, index):
        img = Image.open(self.img_dir + self.img_names[index])
        
        if self.transform is not None:
            img = self.transform(img)
        
        label = self.y[index]
        return img, label

    # defining the len function
    def __len__(self):
        return self.y.shape[0]

In [ ]:
# loading the dataset
train_dataset = EmergencyDataset(csv_path='/content/Dataset/emergency_classification.csv',
                              img_dir='/content/Dataset/images/',
                              transform=preprocessing)

In [ ]:
# using the defined dataset to load data in batch using Dataloader
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=32,
                          shuffle=True)

In [ ]:
# getting the first batch
for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
  break

In [ ]:
# shape of the image and label
batch_X.shape, batch_y.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

## 3. Load weights of pretrained model

In [ ]:
# define model architecture along with pretrained weights of googlenet / inception_v1
googlenet_model = googlenet(pretrained=True)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth


In [ ]:
# print architecture of googlenet
googlenet_model

In [ ]:
# architecture in form of a list
list(googlenet_model.children())

In [ ]:
# removing the fully connected layers
list(googlenet_model.children())[:-3]

In [17]:
# defining the class to extract features
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.net = googlenet(pretrained=True)
        #push to cuda
        if torch.cuda.is_available():
            self.net = self.net.cuda()
        for p in self.net.parameters():
            p.requires_grad = False
        # Define which layers you are going to extract
        self.features = nn.Sequential(*list(self.net.children())[:-3])        

    def forward(self, x):
        return self.features(x)

In [19]:
# prepare input
input = batch_X[:2]
input = input.cuda()

# pass the input to vgg16
if __name__ == "__main__":
    fe = FeatureExtractor()
    output = fe(input)

# shape of the output
output.shape

torch.Size([2, 1024, 7, 7])

## 4. Fine tune the model for the current problem
Steps:-
1. Extract the features
2. Flatten the data
3. Define a Neural Network Model
4. Compile the model
5. Train the model


### 4.1 Extract the features

In [20]:
# extract features using pretrained model 

# create an empty array to store features
features = []
target = []
time_elapsed = []

# set model to eval
googlenet_model.eval()

#deactivates autograd
with torch.no_grad():

  # getting the data in batches using defined data loader
  for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
    if torch.cuda.is_available():
        batch_X = batch_X.cuda()
    
    # to record the time for extracting features
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    start.record()
    # extracting features
    if __name__ == "__main__":
      
        fe = FeatureExtractor()
        batch_features = fe(batch_X)
    
    end.record()

    # Waits for everything to finish running
    torch.cuda.synchronize()

    time_elapsed.append(start.elapsed_time(end))
    #converting to numpy
    batch_features = batch_features.data.cpu().numpy()

    # append in list
    features.append(batch_features)
    target.append(batch_y)
    
    
#save to the array
features = np.concatenate(features, axis=0)
target = np.concatenate(target, axis=0)

In [21]:
# time taken to extract features
print('Time taken in seconds: ', torch.sum(torch.tensor(time_elapsed))*0.001)

Time taken in seconds:  tensor(12.7645)


In [22]:
# shape of the features
features.shape

(2352, 1024, 7, 7)

### 4.2 Flatten the data


In [23]:
#flattening the features
features = features.reshape(len(features),-1) 
features.shape

(2352, 50176)

In [ ]:
# creating the training and validation data
X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.3, stratify=target, random_state=42)

In [ ]:
# shape of training and validation set
(X_train.shape, y_train.shape), (X_valid.shape, y_valid.shape)

(((1646, 50176), (1646,)), ((706, 50176), (706,)))

In [ ]:
# converting training and validation set to PyTorch tensor
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)

X_valid = torch.FloatTensor(X_valid)
y_valid = torch.FloatTensor(y_valid)

### 4.3 Define a Neural Network Model


In [ ]:
# defining the model architecture
model = Sequential(Linear(1024 * 7 * 7, 64),
                   ReLU(),
                   Linear(64, 1),
                   Sigmoid()
                   )

In [ ]:
  # summary of the model
  model

Sequential(
  (0): Linear(in_features=50176, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
# pass an input to the model to understand the output
model(X_train[0].view(1,1024*7*7))

tensor([[0.5117]], grad_fn=<SigmoidBackward>)

## 4.4. Compile the model 

In [ ]:
# define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.BCELoss()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

### 4.5 Train the model


In [ ]:
#define metric
def binary_accuracy(preds, y):
  
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)

    #no. of correctly classified    
    correct = (rounded_preds == y).float()

    #compute accuracy 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
# define training function
def train(X,y,batch_size):

  #activate training phase
  model.train()
  
  #initialization
  epoch_loss, epoch_acc= 0, 0
  no_of_batches = 0

  #randomly create indices
  indices= torch.randperm(len(X))
  
  #loading in batches
  for i in range(0,len(indices),batch_size):
    
    #indices for a batch
    ind = indices[i:i+batch_size]
  
    #batch  
    batch_x=X[ind]
    batch_y=y[ind]
    
    #push to cuda
    if torch.cuda.is_available():
        batch_x, batch_y = batch_x.cuda(), batch_y.cuda()

    #clear gradients
    optimizer.zero_grad()
          
    #forward pass
    outputs = model(batch_x)

    #converting to a 1 dimensional tensor
    outputs = outputs.squeeze()

    #calculate loss and accuracy
    loss = criterion(outputs, batch_y)
    acc = binary_accuracy(outputs, batch_y)  
    
    #Backward pass
    loss.backward()
    
    #Update weights
    optimizer.step()

    #Keep track of the loss and accuracy of a epoch
    epoch_loss = epoch_loss + loss.item()
    epoch_acc  = epoch_acc  + acc.item()

    #No. of batches
    no_of_batches = no_of_batches+1

  return epoch_loss/no_of_batches, epoch_acc/no_of_batches

In [ ]:
# define evaluation function
def evaluate(X,y,batch_size):

  #deactivate training phase
  model.eval()

  #initialization
  epoch_loss, epoch_acc= 0, 0
  no_of_batches = 0

  #randomly create indices
  indices= torch.randperm(len(X))

  #deactivates autograd
  with torch.no_grad():
    
    #loading in batches
    for i in range(0,len(indices),batch_size):
      
      #indices for a batch
      ind = indices[i:i+batch_size]
  
      #batch  
      batch_x= X[ind]
      batch_y= y[ind]

      #push to cuda
      if torch.cuda.is_available():
          batch_x, batch_y = batch_x.cuda(), batch_y.cuda()
        
      #Forward pass
      outputs = model(batch_x)

      #converting the output to 1 Dimensional tensor
      outputs = outputs.squeeze()

      # Calculate loss and accuracy
      loss = criterion(outputs, batch_y)
      acc = binary_accuracy(outputs, batch_y)   
      
      #keep track of loss and accuracy of an epoch
      epoch_loss = epoch_loss + loss.item()
      epoch_acc  = epoch_acc  + acc.item()

      #no. of batches
      no_of_batches = no_of_batches + 1

    return epoch_loss/no_of_batches, epoch_acc/no_of_batches

In [ ]:
# define prediction function
def predict(X,batch_size):
  
  #deactivate training phase
  model.eval()

  # initialization 
  predictions = []

  # create indices
  indices = torch.arange(len(X))

  #deactivates autograd
  with torch.no_grad():
      
      for i in range(0, len(X), batch_size):
        
        #indices for a batch
        ind = indices[i:i+batch_size]

        # batch
        batch_x = X[ind]

        #push to cuda
        if torch.cuda.is_available():
            batch_x = batch_x.cuda()

        #Forward pass
        outputs = model(batch_x)

        #converting the output to 1 Dimensional tensor
        outputs = outputs.squeeze()

        # convert to numpy array
        prediction = outputs.data.cpu().numpy()
        predictions.append(prediction)
    
  # convert to single numpy array
  predictions = np.concatenate(predictions, axis=0)
    
  return predictions

In [ ]:
N_EPOCHS = 10
batch_size = 32

# intialization
best_valid_acc = 0

for epoch in range(N_EPOCHS):
     
    #train the model
    train_loss, train_acc  = train(X_train, y_train, batch_size)
    
    #evaluate the model
    valid_loss, valid_acc = evaluate(X_valid, y_valid, batch_size)

    print('\nEpoch :',epoch,
          'Training loss:',round(train_loss,4),
          '\tTrain Accuracy:',round(train_acc,4),
          '\tValidation loss:',round(valid_loss,4),
          '\tValidation Accuracy:',round(valid_acc,4))

    #save the best model
    if best_valid_acc <= valid_acc:
        best_valid_acc = valid_acc
        torch.save(model.state_dict(), 'saved_weights.pt') 
        print("\n----------------------------------------------------Saved best model------------------------------------------------------------------")   


Epoch : 0 Training loss: 0.4265 	Train Accuracy: 0.8562 	Validation loss: 0.2069 	Validation Accuracy: 0.9212

----------------------------------------------------Saved best model------------------------------------------------------------------

Epoch : 1 Training loss: 0.0836 	Train Accuracy: 0.9692 	Validation loss: 0.2327 	Validation Accuracy: 0.9185

Epoch : 2 Training loss: 0.0327 	Train Accuracy: 0.994 	Validation loss: 0.2204 	Validation Accuracy: 0.9185

Epoch : 3 Training loss: 0.0112 	Train Accuracy: 0.9988 	Validation loss: 0.256 	Validation Accuracy: 0.9198

Epoch : 4 Training loss: 0.0108 	Train Accuracy: 0.9988 	Validation loss: 0.2511 	Validation Accuracy: 0.9293

----------------------------------------------------Saved best model------------------------------------------------------------------

Epoch : 5 Training loss: 0.0071 	Train Accuracy: 0.9988 	Validation loss: 0.2627 	Validation Accuracy: 0.9253

Epoch : 6 Training loss: 0.0063 	Train Accuracy: 0.9988 	Valida

## 5. Validate if it works fine, iterate again if it does not

In [ ]:
 #load weights of best model
path='saved_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [ ]:
valid_loss, valid_accuracy = evaluate(X_valid,y_valid,batch_size)

print("Validation Accuracy:",(valid_accuracy)*100)

Validation Accuracy: 92.93478260869566
